# Structured Streaming using the Python DataFrames API

Apache Spark includes a high-level stream processing API, [Structured Streaming](http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html). In this notebook we take a quick look at how to use the DataFrame API to build Structured Streaming applications. We want to compute real-time metrics like running counts and windowed counts on a stream of timestamped actions (e.g. Open, Close, etc).

To run this notebook, import it and attach it to a Spark cluster.

## Sample Data
We have some sample action data as files in `/databricks-datasets/structured-streaming/events/` which we are going to use to build this appication. Let's take a look at the contents of this directory.

In [0]:
%fs ls /databricks-datasets/structured-streaming/events/

path name size dbfs:/databricks-datasets/structured-streaming/events/file-0.json file-0.json 72530 dbfs:/databricks-datasets/structured-streaming/events/file-1.json file-1.json 72961 dbfs:/databricks-datasets/structured-streaming/events/file-10.json file-10.json 73025 dbfs:/databricks-datasets/structured-streaming/events/file-11.json file-11.json 72999 dbfs:/databricks-datasets/structured-streaming/events/file-12.json file-12.json 72987 dbfs:/databricks-datasets/structured-streaming/events/file-13.json file-13.json 73006 dbfs:/databricks-datasets/structured-streaming/events/file-14.json file-14.json 73003 dbfs:/databricks-datasets/structured-streaming/events/file-15.json file-15.json 73007 dbfs:/databricks-datasets/structured-streaming/events/file-16.json file-16.json 72978 dbfs:/databricks-datasets/structured-streaming/events/file-17.json file-17.json 73008 dbfs:/databricks-datasets/structured-streaming/events/file-18.json file-18.json 73002 dbfs:/databricks-datasets/structured-streaming/events/file-19.json file-19.json 73014 dbfs:/databricks-datasets/structured-streaming/events/file-2.json file-2.json 73007 dbfs:/databricks-datasets/structured-streaming/events/file-20.json file-20.json 72987 dbfs:/databricks-datasets/structured-streaming/events/file-21.json file-21.json 72983 dbfs:/databricks-datasets/structured-streaming/events/file-22.json file-22.json 73009 dbfs:/databricks-datasets/structured-streaming/events/file-23.json file-23.json 72985 dbfs:/databricks-datasets/structured-streaming/events/file-24.json file-24.json 73020 dbfs:/databricks-datasets/structured-streaming/events/file-25.json file-25.json 72980 dbfs:/databricks-datasets/structured-streaming/events/file-26.json file-26.json 73002 dbfs:/databricks-datasets/structured-streaming/events/file-27.json file-27.json 73013 dbfs:/databricks-datasets/structured-streaming/events/file-28.json file-28.json 73005 dbfs:/databricks-datasets/structured-streaming/events/file-29.json file-29.json 72977 dbfs:/databricks-datasets/structured-streaming/events/file-3.json file-3.json 72996 dbfs:/databricks-datasets/structured-streaming/events/file-30.json file-30.json 73009 dbfs:/databricks-datasets/structured-streaming/events/file-31.json file-31.json 73008 dbfs:/databricks-datasets/structured-streaming/events/file-32.json file-32.json 72982 dbfs:/databricks-datasets/structured-streaming/events/file-33.json file-33.json 73033 dbfs:/databricks-datasets/structured-streaming/events/file-34.json file-34.json 72985 dbfs:/databricks-datasets/structured-streaming/events/file-35.json file-35.json 72974 dbfs:/databricks-datasets/structured-streaming/events/file-36.json file-36.json 73013 dbfs:/databricks-datasets/structured-streaming/events/file-37.json file-37.json 72989 dbfs:/databricks-datasets/structured-streaming/events/file-38.json file-38.json 72999 dbfs:/databricks-datasets/structured-streaming/events/file-39.json file-39.json 73013 dbfs:/databricks-datasets/structured-streaming/events/file-4.json file-4.json 72992 dbfs:/databricks-datasets/structured-streaming/events/file-40.json file-40.json 72986 dbfs:/databricks-datasets/structured-streaming/events/file-41.json file-41.json 73019 dbfs:/databricks-datasets/structured-streaming/events/file-42.json file-42.json 72986 dbfs:/databricks-datasets/structured-streaming/events/file-43.json file-43.json 72990 dbfs:/databricks-datasets/structured-streaming/events/file-44.json file-44.json 73018 dbfs:/databricks-datasets/structured-streaming/events/file-45.json file-45.json 72997 dbfs:/databricks-datasets/structured-streaming/events/file-46.json file-46.json 72991 dbfs:/databricks-datasets/structured-streaming/events/file-47.json file-47.json 73009 dbfs:/databricks-datasets/structured-streaming/events/file-48.json file-48.json 72993 dbfs:/databricks-datasets/structured-streaming/events/file-49.json file-49.json 73496 dbfs:/databricks-datasets/structured-streaming/events/file-5.json file-5.json 72998 dbfs:/databricks-datasets/structured-stre

There are about 50 JSON files in the directory. Let's see what each JSON file contains.

In [0]:
%fs head /databricks-datasets/structured-streaming/events/file-0.json

[Truncated to first 65536 bytes]
{"time":1469501107,"action":"Open"}
{"time":1469501147,"action":"Open"}
{"time":1469501202,"action":"Open"}
{"time":1469501219,"action":"Open"}
{"time":1469501225,"action":"Open"}
{"time":1469501234,"action":"Open"}
{"time":1469501245,"action":"Open"}
{"time":1469501246,"action":"Open"}
{"time":1469501248,"action":"Open"}
{"time":1469501256,"action":"Open"}
{"time":1469501264,"action":"Open"}
{"time":1469501266,"action":"Open"}
{"time":1469501267,"action":"Open"}
{"time":1469501269,"action":"Open"}
{"time":1469501271,"action":"Open"}
{"time":1469501282,"action":"Open"}
{"time":1469501285,"action":"Open"}
{"time":1469501291,"action":"Open"}
{"time":1469501297,"action":"Open"}
{"time":1469501303,"action":"Open"}
{"time":1469501322,"action":"Open"}
{"time":1469501335,"action":"Open"}
{"time":1469501344,"action":"Open"}
{"time":1469501346,"action":"Open"}
{"time":1469501349,"action":"Open"}
{"time":1469501357,"action":"Open"}
{"time":1469501366,"action":"Open"}
{"time":1469501371,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501381,"action":"Open"}
{"time":1469501392,"action":"Open"}
{"time":1469501402,"action":"Open"}
{"time":1469501407,"action":"Open"}
{"time":1469501410,"action":"Open"}
{"time":1469501420,"action":"Open"}
{"time":1469501424,"action":"Open"}
{"time":1469501438,"action":"Open"}
{"time":1469501442,"action":"Close"}
{"time":1469501462,"action":"Open"}
{"time":1469501480,"action":"Open"}
{"time":1469501488,"action":"Open"}
{"time":1469501489,"action":"Open"}
{"time":1469501491,"action":"Open"}
{"time":1469501503,"action":"Open"}
{"time":1469501505,"action":"Open"}
{"time":1469501509,"action":"Open"}
{"time":1469501513,"action":"Open"}
{"time":1469501517,"action":"Open"}
{"time":1469501520,"action":"Open"}
{"time":1469501525,"action":"Open"}
{"time":1469501533,"action":"Open"}
{"time":1469501539,"action":"Open"}
{"time":1469501540,"action":"Open"}
{"time":1469501541,"action":"Open"}
{"time":1469501543,"action":"Open"}
{"time":1469501544,"action":"Open"}
{"time":1469501545,"action":"Close"}
{"time":1469501545,"action":"Open"}
{"time":1469501547,"action":"Open"}
{"time":1469501552,"action":"Open"}
{"time":1469501557,"action":"Open"}
{"time":1469501559,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501565,"action":"Open"}
{"time":1469501566,"action":"Open"}
{"time":1469501574,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501578,"action":"Open"}
{"time":1469501581,"action":"Open"}
{"time":1469501584,"action":"Open"}
{"time":1469501600,"action":"Open"}
{"time":1469501601,"action":"Open"}
{"time":1469501603,"action":"Open"}
{"time":1469501610,"action":"Open"}
{"time":1469501620,"action":"Open"}
{"time":1469501621,"action":"Open"}
{"time":1469501625,"action":"Open"}
{"time":1469501625,"action":"Close"}
{"time":1469501626,"action":"Open"}
{"time":1469501631,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501638,"action":"Open"}
{"time":1469501643,"action":"Open"}
{"time":1469501646,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501663,"action":"Open"}
{"time":1469501667,"action":"Open"}
{"time":1469501674,"action":"Open"}
{"time":1469501675,"action":"Open"}
{"time":1469501678,"action":"Close"}
{"time":1469501680,"action":"Open"}
{"time":1469501685,"action":"Open"}
{"time":1469501686,"action":"Open"}
{"time":1469501689,"action":"Open"}
{"time":1469501691,"action":"Open"}
{"time":1469501694,"action":"Open"}
{"time":1469501696,"action":"Close"}
{"time":1469501702,"action":"Open"}
{"time":1469501703,"action":"Open"}
{"time":1469501704,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501710,"action":"Open"}
{"time":1469501715,"action":"Open"}
{"

Each line in the file contains JSON record with two fields - `time` and `action`. Let's try to analyze these files interactively.

## Batch/Interactive Processing
The usual first step in attempting to process the data is to interactively query the data. Let's define a static DataFrame on the files, and give it a table name.

In [0]:
from pyspark.sql.types import *

inputPath = "/databricks-datasets/structured-streaming/events/"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])

# Static DataFrame representing data in the JSON files
staticInputDF = (
  spark
    .read
    .schema(jsonSchema)
    .json(inputPath)
)

display(staticInputDF)

time action 2016-07-28T04:19:28.000+0000 Close 2016-07-28T04:19:28.000+0000 Close 2016-07-28T04:19:29.000+0000 Open 2016-07-28T04:19:31.000+0000 Close 2016-07-28T04:19:31.000+0000 Open 2016-07-28T04:19:31.000+0000 Open 2016-07-28T04:19:32.000+0000 Close 2016-07-28T04:19:33.000+0000 Close 2016-07-28T04:19:35.000+0000 Close 2016-07-28T04:19:36.000+0000 Open 2016-07-28T04:19:38.000+0000 Close 2016-07-28T04:19:40.000+0000 Open 2016-07-28T04:19:41.000+0000 Close 2016-07-28T04:19:42.000+0000 Open 2016-07-28T04:19:45.000+0000 Open 2016-07-28T04:19:47.000+0000 Open 2016-07-28T04:19:48.000+0000 Open 2016-07-28T04:19:49.000+0000 Open 2016-07-28T04:19:55.000+0000 Open 2016-07-28T04:20:00.000+0000 Close 2016-07-28T04:20:00.000+0000 Open 2016-07-28T04:20:01.000+0000 Open 2016-07-28T04:20:03.000+0000 Close 2016-07-28T04:20:07.000+0000 Open 2016-07-28T04:20:11.000+0000 Open 2016-07-28T04:20:12.000+0000 Close 2016-07-28T04:20:12.000+0000 Open 2016-07-28T04:20:13.000+0000 Close 2016-07-28T04:20:16.000+0000 Open 2016-07-28T04:20:23.000+0000 Close 2016-07-28T04:20:23.000+0000 Close 2016-07-28T04:20:23.000+0000 Open 2016-07-28T04:20:26.000+0000 Close 2016-07-28T04:20:30.000+0000 Close 2016-07-28T04:20:32.000+0000 Open 2016-07-28T04:20:32.000+0000 Open 2016-07-28T04:20:34.000+0000 Close 2016-07-28T04:20:36.000+0000 Open 2016-07-28T04:20:42.000+0000 Close 2016-07-28T04:20:42.000+0000 Open 2016-07-28T04:20:42.000+0000 Open 2016-07-28T04:20:48.000+0000 Close 2016-07-28T04:20:48.000+0000 Close 2016-07-28T04:20:48.000+0000 Open 2016-07-28T04:20:50.000+0000 Open 2016-07-28T04:20:52.000+0000 Open 2016-07-28T04:20:55.000+0000 Open 2016-07-28T04:20:55.000+0000 Open 2016-07-28T04:20:56.000+0000 Close 2016-07-28T04:20:56.000+0000 Close 2016-07-28T04:20:56.000+0000 Open 2016-07-28T04:20:59.000+0000 Open 2016-07-28T04:20:59.000+0000 Open 2016-07-28T04:21:02.000+0000 Close 2016-07-28T04:21:04.000+0000 Open 2016-07-28T04:21:08.000+0000 Open 2016-07-28T04:21:11.000+0000 Close 2016-07-28T04:21:13.000+0000 Open 2016-07-28T04:21:18.000+0000 Open 2016-07-28T04:21:19.000+0000 Open 2016-07-28T04:21:20.000+0000 Close 2016-07-28T04:21:22.000+0000 Open 2016-07-28T04:21:23.000+0000 Open 2016-07-28T04:21:27.000+0000 Close 2016-07-28T04:21:28.000+0000 Open 2016-07-28T04:21:31.000+0000 Close 2016-07-28T04:21:32.000+0000 Open 2016-07-28T04:21:32.000+0000 Open 2016-07-28T04:21:33.000+0000 Open 2016-07-28T04:21:34.000+0000 Close 2016-07-28T04:21:34.000+0000 Open 2016-07-28T04:21:35.000+0000 Close 2016-07-28T04:21:37.000+0000 Open 2016-07-28T04:21:38.000+0000 Close 2016-07-28T04:21:44.000+0000 Open 2016-07-28T04:21:46.000+0000 Close 2016-07-28T04:21:46.000+0000 Open 2016-07-28T04:21:48.000+0000 Close 2016-07-28T04:21:49.000+0000 Open 2016-07-28T04:21:50.000+0000 Close 2016-07-28T04:21:52.000+0000 Close 2016-07-28T04:21:52.000+0000 Open 2016-07-28T04:21:52.000+0000 Open 2016-07-28T04:21:53.000+0000 Open 2016-07-28T04:21:53.000+0000 Open 2016-07-28T04:21:56.000+0000 Close 2016-07-28T04:21:56.000+0000 Close 2016-07-28T04:21:57.000+0000 Close 2016-07-28T04:21:58.000+0000 Open 2016-07-28T04:21:59.000+0000 Close 2016-07-28T04:22:01.000+0000 Open 2016-07-28T04:22:06.000+0000 Close 2016-07-28T04:22:10.000+0000 Open 2016-07-28T04:22:11.000+0000 Close 2016-07-28T04:22:12.000+0000 Open 2016-07-28T04:22:14.000+0000 Close 2016-07-28T04:22:15.000+0000 Close 2016-07-28T04:22:15.000+0000 Open 2016-07-28T04:22:19.000+0000 Close 2016-07-28T04:22:20.000+0000 Close 2016-07-28T04:22:24.000+0000 Close 2016-07-28T04:22:24.000+0000 Close 2016-07-28T04:22:24.000+0000 Open 2016-07-28T04:22:30.000+0000 Open 2016-07-28T04:22:31.000+0000 Close 2016-07-28T04:22:31.000+0000 Open 2016-07-28T04:22:33.000+0000 Close 2016-07-28T04:22:39.000+0000 Open 2016-07-28T04:22:40.000+0000 Close 2016-07-28T04:22:43.000+0000 Open 2016-07-28T04:22:44.000+0000 Open 2016-07-28T04:22:49.000+0000 Close 2016-07-28T04:22:50.000+0000 Open 2016-07-28T04:22:55.000+0000 Open 2016-07-28T04:22:56.000+0000 Open 2016-07-28T04:22:58.000+0000

Now we can compute the number of "open" and "close" actions with one hour windows. To do this, we will group by the `action` column and 1 hour windows over the `time` column.

In [0]:
from pyspark.sql.functions import *      # for window() function

staticCountsDF = (
  staticInputDF
    .groupBy(
       staticInputDF.action, 
       window(staticInputDF.time, "1 hour"))    
    .count()
)
staticCountsDF.cache()

# Register the DataFrame as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")

In [0]:
staticCountsDF.show()

+------+--------------------+-----+
action| window|count|
+------+--------------------+-----+
 Close|{2016-07-26 13:00...| 1028|
 Open|{2016-07-26 18:00...| 1004|
 Close|{2016-07-27 02:00...| 971|
 Open|{2016-07-27 04:00...| 995|
 Open|{2016-07-27 05:00...| 986|
 Open|{2016-07-26 05:00...| 1000|
 Open|{2016-07-26 11:00...| 991|
 Close|{2016-07-26 06:00...| 1011|
 Close|{2016-07-27 05:00...| 987|
 Open|{2016-07-26 10:00...| 1007|
 Close|{2016-07-26 16:00...| 984|
 Close|{2016-07-26 04:00...| 815|
 Close|{2016-07-27 03:00...| 1025|
 Open|{2016-07-27 07:00...| 998|
 Close|{2016-07-27 13:00...| 986|
 Open|{2016-07-27 18:00...| 995|
 Close|{2016-07-26 12:00...| 960|
 Close|{2016-07-27 08:00...| 1024|
 Close|{2016-07-27 01:00...| 1013|
 Open|{2016-07-27 19:00...| 1007|
+------+--------------------+-----+
only showing top 20 rows

Now we can directly use SQL to query the table. For example, here are the total counts across all the hours.

In [0]:
%sql select action, sum(count) as total_count from static_counts group by action

action total_count Close 50000 Open 50000

How about a timeline of windowed counts?

In [0]:
%sql
select
  action,
  date_format(window.end,
  "MMM-dd HH:mm" 
) as time,
count
from
  static_counts order by
  time,
  action

action time count Close Jul-26 03:00 11 Open Jul-26 03:00 179 Close Jul-26 04:00 344 Open Jul-26 04:00 1001 Close Jul-26 05:00 815 Open Jul-26 05:00 999 Close Jul-26 06:00 1003 Open Jul-26 06:00 1000 Close Jul-26 07:00 1011 Open Jul-26 07:00 993 Close Jul-26 08:00 989 Open Jul-26 08:00 1008 Close Jul-26 09:00 985 Open Jul-26 09:00 996 Close Jul-26 10:00 983 Open Jul-26 10:00 1000 Close Jul-26 11:00 1022 Open Jul-26 11:00 1007 Close Jul-26 12:00 1028 Open Jul-26 12:00 991 Close Jul-26 13:00 960 Open Jul-26 13:00 996 Close Jul-26 14:00 1028 Open Jul-26 14:00 1006 Close Jul-26 15:00 994 Open Jul-26 15:00 991 Close Jul-26 16:00 988 Open Jul-26 16:00 1020 Close Jul-26 17:00 984 Open Jul-26 17:00 992 Close Jul-26 18:00 1036 Open Jul-26 18:00 990 Close Jul-26 19:00 1001 Open Jul-26 19:00 1004 Close Jul-26 20:00 967 Open Jul-26 20:00 998 Close Jul-26 21:00 1035 Open Jul-26 21:00 1010 Close Jul-26 22:00 995 Open Jul-26 22:00 998 Close Jul-26 23:00 1036 Open Jul-26 23:00 997 Close Jul-27 00:00 950 Open Jul-27 00:00 1000 Close Jul-27 01:00 1008 Open Jul-27 01:00 998 Close Jul-27 02:00 1013 Open Jul-27 02:00 1004 Close Jul-27 03:00 971 Open Jul-27 03:00 992 Close Jul-27 04:00 1025 Open Jul-27 04:00 1014 Close Jul-27 05:00 989 Open Jul-27 05:00 995 Close Jul-27 06:00 987 Open Jul-27 06:00 986 Close Jul-27 07:00 1026 Open Jul-27 07:00 1016 Close Jul-27 08:00 982 Open Jul-27 08:00 998 Close Jul-27 09:00 1024 Open Jul-27 09:00 1002 Close Jul-27 10:00 990 Open Jul-27 10:00 992 Close Jul-27 11:00 1001 Open Jul-27 11:00 1006 Close Jul-27 12:00 1006 Open Jul-27 12:00 998 Close Jul-27 13:00 1035 Open Jul-27 13:00 994 Close Jul-27 14:00 986 Open Jul-27 14:00 1008 Close Jul-27 15:00 948 Open Jul-27 15:00 984 Close Jul-27 16:00 1018 Open Jul-27 16:00 1017 Close Jul-27 17:00 970 Open Jul-27 17:00 992 Close Jul-27 18:00 1020 Open Jul-27 18:00 1007 Close Jul-27 19:00 1036 Open Jul-27 19:00 995 Close Jul-27 20:00 969 Open Jul-27 20:00 1007 Close Jul-27 21:00 1025 Open Jul-27 21:00 1005 Close Jul-27 22:00 979 Open Jul-27 22:00 998 Close Jul-27 23:00 996 Open Jul-27 23:00 986 Close Jul-28 00:00 1011 Open Jul-28 00:00 1008 Close Jul-28 01:00 988 Open Jul-28 01:00 1000 Close Jul-28 02:00 1010 Open Jul-28 02:00 1001 Close Jul-28 03:00 1007 Open Jul-28 03:00 1000 Close Jul-28 04:00 993 Open Jul-28 04:00 996 Close Jul-28 05:00 960 Open Jul-28 05:00 825 Close Jul-28 06:00 671 Close Jul-28 07:00 191

In [0]:
newdf = spark.sql('''select
  action,
  date_format(window.end,
  "MMM-dd HH:mm" 
) as time,
count
from
  static_counts order by
  time,
  action''')

In [0]:
newdf.show()

+------+------------+-----+
action| time|count|
+------+------------+-----+
 Close|Jul-26 03:00| 11|
 Open|Jul-26 03:00| 179|
 Close|Jul-26 04:00| 344|
 Open|Jul-26 04:00| 1001|
 Close|Jul-26 05:00| 815|
 Open|Jul-26 05:00| 999|
 Close|Jul-26 06:00| 1003|
 Open|Jul-26 06:00| 1000|
 Close|Jul-26 07:00| 1011|
 Open|Jul-26 07:00| 993|
 Close|Jul-26 08:00| 989|
 Open|Jul-26 08:00| 1008|
 Close|Jul-26 09:00| 985|
 Open|Jul-26 09:00| 996|
 Close|Jul-26 10:00| 983|
 Open|Jul-26 10:00| 1000|
 Close|Jul-26 11:00| 1022|
 Open|Jul-26 11:00| 1007|
 Close|Jul-26 12:00| 1028|
 Open|Jul-26 12:00| 991|
+------+------------+-----+
only showing top 20 rows

Note the two ends of the graph. The close actions are generated such that they are after the corresponding open actions, so there are more "opens" in the beginning and more "closes" in the end.

## Stream Processing 
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [0]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.action, 
      window(streamingInputDF.time, "1 hour"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

True

As you can see, `streamingCountsDF` is a streaming Dataframe (`streamingCountsDF.isStreaming` was `true`). You can start streaming computation, by defining the sink and starting it. 
In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table.

In [0]:

spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table 
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

In [0]:
%sql

select * from counts

action window count Open List(2016-07-26T12:00:00.000+0000, 2016-07-26T13:00:00.000+0000) 996 Close List(2016-07-26T16:00:00.000+0000, 2016-07-26T17:00:00.000+0000) 984 Close List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000) 815 Close List(2016-07-26T07:00:00.000+0000, 2016-07-26T08:00:00.000+0000) 989 Close List(2016-07-26T06:00:00.000+0000, 2016-07-26T07:00:00.000+0000) 1011 Close List(2016-07-26T15:00:00.000+0000, 2016-07-26T16:00:00.000+0000) 988 Open List(2016-07-26T13:00:00.000+0000, 2016-07-26T14:00:00.000+0000) 1006 Close List(2016-07-26T14:00:00.000+0000, 2016-07-26T15:00:00.000+0000) 994 Open List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000) 179 Open List(2016-07-26T09:00:00.000+0000, 2016-07-26T10:00:00.000+0000) 1000 Close List(2016-07-26T08:00:00.000+0000, 2016-07-26T09:00:00.000+0000) 985 Close List(2016-07-26T19:00:00.000+0000, 2016-07-26T20:00:00.000+0000) 331 Close List(2016-07-26T05:00:00.000+0000, 2016-07-26T06:00:00.000+0000) 1003 Open List(2016-07-26T07:00:00.000+0000, 2016-07-26T08:00:00.000+0000) 1008 Close List(2016-07-26T13:00:00.000+0000, 2016-07-26T14:00:00.000+0000) 1028 Close List(2016-07-26T17:00:00.000+0000, 2016-07-26T18:00:00.000+0000) 1036 Open List(2016-07-26T14:00:00.000+0000, 2016-07-26T15:00:00.000+0000) 991 Close List(2016-07-26T10:00:00.000+0000, 2016-07-26T11:00:00.000+0000) 1022 Close List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000) 11 Close List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000) 344 Open List(2016-07-26T10:00:00.000+0000, 2016-07-26T11:00:00.000+0000) 1007 Open List(2016-07-26T08:00:00.000+0000, 2016-07-26T09:00:00.000+0000) 996 Open List(2016-07-26T05:00:00.000+0000, 2016-07-26T06:00:00.000+0000) 1000 Close List(2016-07-26T09:00:00.000+0000, 2016-07-26T10:00:00.000+0000) 983 Open List(2016-07-26T06:00:00.000+0000, 2016-07-26T07:00:00.000+0000) 993 Open List(2016-07-26T15:00:00.000+0000, 2016-07-26T16:00:00.000+0000) 1020 Open List(2016-07-26T19:00:00.000+0000, 2016-07-26T20:00:00.000+0000) 314 Close List(2016-07-26T12:00:00.000+0000, 2016-07-26T13:00:00.000+0000) 960 Close List(2016-07-26T18:00:00.000+0000, 2016-07-26T19:00:00.000+0000) 1001 Open List(2016-07-26T17:00:00.000+0000, 2016-07-26T18:00:00.000+0000) 990 Close List(2016-07-26T11:00:00.000+0000, 2016-07-26T12:00:00.000+0000) 1028 Open List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000) 1001 Open List(2016-07-26T18:00:00.000+0000, 2016-07-26T19:00:00.000+0000) 1004 Open List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000) 999 Open List(2016-07-26T11:00:00.000+0000, 2016-07-26T12:00:00.000+0000) 991 Open List(2016-07-26T16:00:00.000+0000, 2016-07-26T17:00:00.000+0000) 992

`query` is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts. 

Note the status of query in the above cell. The progress bar shows that the query is active. 
Furthermore, if you expand the `> counts` above, you will find the number of files they have already processed. 

Let's wait a bit for a few files to be processed and then interactively query the in-memory `counts` table.

In [0]:
from time import sleep
sleep(5)  # wait a bit for computation to start

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action time count Close Jul-26 03:00 11 Open Jul-26 03:00 179 Close Jul-26 04:00 344 Open Jul-26 04:00 1001 Close Jul-26 05:00 815 Open Jul-26 05:00 999 Close Jul-26 06:00 1003 Open Jul-26 06:00 1000 Close Jul-26 07:00 1011 Open Jul-26 07:00 993 Close Jul-26 08:00 989 Open Jul-26 08:00 1008 Close Jul-26 09:00 985 Open Jul-26 09:00 996 Close Jul-26 10:00 983 Open Jul-26 10:00 1000 Close Jul-26 11:00 1022 Open Jul-26 11:00 1007 Close Jul-26 12:00 1028 Open Jul-26 12:00 991 Close Jul-26 13:00 960 Open Jul-26 13:00 996 Close Jul-26 14:00 1028 Open Jul-26 14:00 1006 Close Jul-26 15:00 994 Open Jul-26 15:00 991 Close Jul-26 16:00 988 Open Jul-26 16:00 1020 Close Jul-26 17:00 984 Open Jul-26 17:00 992 Close Jul-26 18:00 1036 Open Jul-26 18:00 990 Close Jul-26 19:00 1001 Open Jul-26 19:00 1004 Close Jul-26 20:00 967 Open Jul-26 20:00 998 Close Jul-26 21:00 1035 Open Jul-26 21:00 1010 Close Jul-26 22:00 995 Open Jul-26 22:00 998 Close Jul-26 23:00 325 Open Jul-26 23:00 317

We see the timeline of windowed counts (similar to the static one earlier) building up. If we keep running this interactive query repeatedly, we will see the latest updated counts which the streaming query is updating in the background.

In [0]:
sleep(5)  # wait a bit more for more data to be computed

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action time count Close Jul-26 03:00 11 Open Jul-26 03:00 179 Close Jul-26 04:00 344 Open Jul-26 04:00 1001 Close Jul-26 05:00 815 Open Jul-26 05:00 999 Close Jul-26 06:00 1003 Open Jul-26 06:00 1000 Close Jul-26 07:00 1011 Open Jul-26 07:00 993 Close Jul-26 08:00 989 Open Jul-26 08:00 1008 Close Jul-26 09:00 985 Open Jul-26 09:00 996 Close Jul-26 10:00 983 Open Jul-26 10:00 1000 Close Jul-26 11:00 1022 Open Jul-26 11:00 1007 Close Jul-26 12:00 1028 Open Jul-26 12:00 991 Close Jul-26 13:00 960 Open Jul-26 13:00 996 Close Jul-26 14:00 1028 Open Jul-26 14:00 1006 Close Jul-26 15:00 994 Open Jul-26 15:00 991 Close Jul-26 16:00 988 Open Jul-26 16:00 1020 Close Jul-26 17:00 984 Open Jul-26 17:00 992 Close Jul-26 18:00 1036 Open Jul-26 18:00 990 Close Jul-26 19:00 1001 Open Jul-26 19:00 1004 Close Jul-26 20:00 967 Open Jul-26 20:00 998 Close Jul-26 21:00 1035 Open Jul-26 21:00 1010 Close Jul-26 22:00 995 Open Jul-26 22:00 998 Close Jul-26 23:00 1036 Open Jul-26 23:00 997 Close Jul-27 00:00 950 Open Jul-27 00:00 1000 Close Jul-27 01:00 1008 Open Jul-27 01:00 998 Close Jul-27 02:00 318 Open Jul-27 02:00 335

In [0]:
sleep(5)  # wait a bit more for more data to be computed

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action time count Close Jul-26 03:00 11 Open Jul-26 03:00 179 Close Jul-26 04:00 344 Open Jul-26 04:00 1001 Close Jul-26 05:00 815 Open Jul-26 05:00 999 Close Jul-26 06:00 1003 Open Jul-26 06:00 1000 Close Jul-26 07:00 1011 Open Jul-26 07:00 993 Close Jul-26 08:00 989 Open Jul-26 08:00 1008 Close Jul-26 09:00 985 Open Jul-26 09:00 996 Close Jul-26 10:00 983 Open Jul-26 10:00 1000 Close Jul-26 11:00 1022 Open Jul-26 11:00 1007 Close Jul-26 12:00 1028 Open Jul-26 12:00 991 Close Jul-26 13:00 960 Open Jul-26 13:00 996 Close Jul-26 14:00 1028 Open Jul-26 14:00 1006 Close Jul-26 15:00 994 Open Jul-26 15:00 991 Close Jul-26 16:00 988 Open Jul-26 16:00 1020 Close Jul-26 17:00 984 Open Jul-26 17:00 992 Close Jul-26 18:00 1036 Open Jul-26 18:00 990 Close Jul-26 19:00 1001 Open Jul-26 19:00 1004 Close Jul-26 20:00 967 Open Jul-26 20:00 998 Close Jul-26 21:00 1035 Open Jul-26 21:00 1010 Close Jul-26 22:00 995 Open Jul-26 22:00 998 Close Jul-26 23:00 1036 Open Jul-26 23:00 997 Close Jul-27 00:00 950 Open Jul-27 00:00 1000 Close Jul-27 01:00 1008 Open Jul-27 01:00 998 Close Jul-27 02:00 1013 Open Jul-27 02:00 1004 Close Jul-27 03:00 971 Open Jul-27 03:00 992 Close Jul-27 04:00 330 Open Jul-27 04:00 343

Also, let's see the total number of "opens" and "closes".

In [0]:
%sql select action, sum(count) as total_count from counts group by action order by action

action total_count Close 37491 Open 38509

If you keep running the above query repeatedly, you will always find that the number of "opens" is more than the number of "closes", as expected in a data stream where a "close" always appear after corresponding "open". This shows that Structured Streaming ensures **prefix integrity**. Read the blog posts linked below if you want to know more.

Note that there are only a few files, so consuming all of them there will be no updates to the counts. Rerun the query if you want to interact with the streaming query again.

Finally, you can stop the query running in the background, either by clicking on the 'Cancel' link in the cell of the query, or by executing `query.stop()`. Either way, when the query is stopped, the status of the corresponding cell above will automatically update to `TERMINATED`.